In [1]:
options(warn=-1)

suppressMessages(library(ArchR))
suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(harmony))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(harmony))
suppressMessages(library(Nebulosa))
suppressMessages(library(ggpubr))
suppressMessages(library(Ipaper))
suppressMessages(library(phateR))
suppressMessages(library(viridis))
suppressMessages(library(parallel))
suppressMessages(library(ggrepel))
suppressMessages(library(circlize))
suppressMessages(library(foreach))

In [2]:
df_grn <- read.csv("../data/snATAC_SCARA5_Myofib/gene_regulatory_network.csv")
head(df_grn)

,tf,gene,correlation
,<chr>,<chr>,<dbl>
1,KLF2,PCOLCE2,0.9462578
2,KLF4,PCOLCE2,0.9476462
3,SP3,PCOLCE2,0.9405444
4,ATF4,PCOLCE2,0.9121518
5,ELF2,PCOLCE2,0.7645778
6,THAP11,PCOLCE2,0.8967850


In [3]:
df_anno <- read.csv("../../../visiumSpatial/metadata/sample_annotation.csv", header = TRUE)

patient_region_id <- df_anno$patient_region_id
patient_region_id <- stringr::str_replace_all(patient_region_id, "/", "_")

names(patient_region_id) <- df_anno$sample_id

In [4]:
for(sample in names(patient_region_id)){
    # load visium object
    obj <- readRDS(glue::glue("../../../visiumSpatial/from_rico/processed_visium/objects/{sample}.rds"))  
    DefaultAssay(obj) <- "Spatial"
    
    obj <- obj %>% NormalizeData() %>%
        ScaleData()
    
    output_name <- patient_region_id[sample]
    
    for(sel_tf in unique(df_grn$tf)){
        tf_gex_dir <- glue::glue("../data/snATAC_SCARA5_Myofib/VisGRNInSpatial/{sel_tf}/TF_Expression")
        tf_target_gex_dir <- glue::glue("../data/snATAC_SCARA5_Myofib/VisGRNInSpatial/{sel_tf}/TF_Target_Expression")

        df_sub <- subset(df_grn, tf == sel_tf)

        target_genes <- list(df_sub$gene)

        if(!dir.exists(tf_gex_dir)){
            dir.create(tf_gex_dir, recursive = TRUE)
        }

        if(!dir.exists(tf_target_gex_dir)){
            dir.create(tf_target_gex_dir, recursive = TRUE)
        }
    
        if(sel_tf %in% rownames(obj)){
            p <- SpatialFeaturePlot(obj, features = sel_tf, max.cutoff = "q95") +
                scale_fill_viridis(option = "B") 

            ggsave(glue::glue("{tf_gex_dir}/{sample}_{output_name}.pdf"), p, 
                   height = 6, width = 6)
      
            # get module score
            obj <- AddModuleScore(obj, features = target_genes, assay = "SCT")
            
            p <- SpatialFeaturePlot(obj, features = "Cluster1", min.cutoff = "q5",
                                    max.cutoff = "q95") +
                    scale_fill_viridis(option = "B") 
        
            ggsave(glue::glue("{tf_target_gex_dir}/{sample}_{output_name}.pdf"), p, 
               height = 6, width = 6)
        
        
        }     
    
    }
    

}

Centering and scaling data matrix

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will replace the existing scale.

Scale for 'fill' is already present. Adding another scale for 'fill', which
will repl

In [5]:
# for(sel_tf in unique(df_grn$tf)){
    
#     tf_gex_dir <- glue::glue("../data/snATAC_healthy_intermediate_damaged/VisGRNInSpatial/{sel_tf}/TF_Expression")
#     tf_target_gex_dir <- glue::glue("../data/snATAC_healthy_intermediate_damaged/VisGRNInSpatial/{sel_tf}/TF_Target_Expression")

#     df_sub <- subset(df_grn, tf == sel_tf)
    
#     target_genes <- list(df_sub$gene)
    
#     if(!dir.exists(tf_gex_dir)){
#         dir.create(tf_gex_dir, recursive = TRUE)
#     }
    
#     if(!dir.exists(tf_target_gex_dir)){
#         dir.create(tf_target_gex_dir, recursive = TRUE)
#     }
    
#     foreach(sample = names(patient_region_id)) %dopar% {
#         # load visium object
#         obj <- readRDS(glue::glue("../../../visiumSpatial/from_rico/processed_visium/objects/{sample}.rds"))
#         DefaultAssay(obj) <- "SCT"
        
#         output_name <- patient_region_id[sample]
        
#         if(sel_tf %in% rownames(obj)){
#             p <- SpatialFeaturePlot(obj, features = sel_tf, max.cutoff = "q95") +
#                 scale_fill_gradientn(colors = ArchR::paletteContinuous("horizonExtra", n = 256))

#             ggsave(glue::glue("{tf_gex_dir}/{sample}_{output_name}.pdf"), p, 
#                    height = 6, width = 6)
      
#             # get module score
#             obj <- AddModuleScore(obj, features = target_genes, assay = "SCT")
            
#             p <- SpatialFeaturePlot(obj, features = "Cluster1", max.cutoff = "q95") +
#             scale_fill_gradientn(colors = ArchR::paletteContinuous("horizonExtra", n = 256))
        
#             ggsave(glue::glue("{tf_target_gex_dir}/{sample}_{output_name}.pdf"), p, 
#                height = 6, width = 6)
        
        
#         }
#     }

# }